# Config file builder

Erik Hogenbirk (ehogenbi@nikhef.nl, hogenbirk91@gmail.com)*

## Imports, settings, functions

In [288]:
# Imports
import json
import os
import numpy as np

In [291]:
baselines_placeholder = {
    '1724' : [8000 for _ in range(8)],
    '1730' : [8000 for _ in range(8)],
}
with open('baselines_placeholder.json', 'w') as outfile:
    json.dump(baselines_placeholder, outfile, indent = 4)

## Class definition

In [303]:
class Config():
    '''
    Class that helps building and storing config files.
    '''
    __version__ = '0.0.0'
    def __init__(self, name, output_dir = '/home/xams/xams/daqcontrol/ini/', 
                 baseline_file = 'baselines_placeholder.json',
                 verbose=False):
        self.name = name
        self.output_dir = output_dir
        self.boards = [1724, 1730]
        self.verbose = verbose
        with open(baseline_file) as f:
            self.baselines = json.load(f)
        self.cf = {
            "strax_config" : {},
            "data_folder" : '/data/xenon/xams/run10/',
            "delete_raw_records" : False,
            "baselines_used" : self.baselines,
            # Links in optical chain
            # HW configuration: the 1730 takes the black cable in bottom of opt link,
            # top goes to bottom of 1724, top of V1724 goes to red fibre
            # that goes back to A3818
            "links": [{
                "type": "V2718",
                "reader": 0,
                "crate": 0,
                "link": 0
            },
            {
                "type": "V2718",
                "reader": 0,
                "crate": 1,
                "link": 0
            },
            ],
            "boards": [
                {
                "crate": 0,
                "serial": "1730",
                "reader": 0,
                "type": "V1724",
                "vme_address": "Whatever?",
                "link": 0,
                },
                {
                "crate": 1,
                "serial": "1724",
                "reader": 0,
                "type": "V1724",
                "vme_address": "Whatever?",
                "link": 0,
                },

            ],
            # blt_size {int} : block transfer size in bytes (decimal).
            # V1724 standard: 524288
            # V1730 maximum: 2097152?
            "blt_size": 524288,
            # run_start {int} : 0 - board internal (run start via register), 
            # 1 - s-in (for synchronized boards)
            "run_start": 0,
            "baseline_mode": 0,
            # write_mode {int}: 0-no writing, 1-to file, 2-mongodb
            "write_mode": 2,

            "mongo": {
                "write_concern": 0,
                "min_insert_size": 1,
                "database": "xamsdata0",
                "address": "mongodb://localhost:27017",
                "collection": "onzin"
            },
            "processing_readout_threshold": 0,
            "processing_num_threads": 8,
            "processing_mode": 3,
            "occurrence_integral": 0,

            "compression": 1,

            "registers": [
                # This resets all before any other settings
                {
                    "comment": "Reset software (all registers)",
                    "register": "EF24",
                    "board": "-1",
                    "value": "1"
                },
                {
                    "comment": "BLT event number",
                    "register": "EF1C",
                    "board": "-1",
                    "value": "1"
                    # Maximum number of events loaded into each block.
                    # 40hex = 64dec
                },
                {
                    "comment": "Enable BERR (error messages)",
                    "register": "EF00",
                    "board": "-1",
                    "value": "10"
                },
                {
                    "comment": "Aquisition control register",
                    "register": "8100",
                    "board": "-1",
                    "value": "5" 
                    #5: s-in control, 0: board control
                },
                {
                    "comment": "Channel calibration",
                    "register": "809C",
                    "board": "-1",
                    "value": "1"
                    # enables channel calibration
                },
                {
                    "comment": "Channel enable mask",
                    "register": "8120",
                    "board": "1724",
                    "value": "FF"
                },
                {
                    "comment": "Channel enable mask",
                    "register": "8120",
                    "board": "1730",
                    "value": "01"
                },
                {
                    "comment": "Trigger source enable mask",
                    "register": "810C",
                    "board": "1730",
                    "value": "C0F00001"
                },
                {
                    "comment": "Trigger source enable mask",
                    "register": "810C",
                    "board": "1724",
                    "value": "C0F00000"
                },
                {
                    "comment": "Buffer organisation",
                    "register": "800C",
                    "board": "1730",
                    "value": "A"
                    # A = 5110 Samples
                },
                {
                    "comment": "Buffer organisation",
                    "register": "800C",
                    "board": "1724",
                    "value": "A"
                    # A = 512 S
                },
                {
                    "comment": "Measurement time after trigger",
                    "register": "8114",
                    "board": "1724",
                    "value": "80"
                    # 80hex = 128 -> 128 * 2 = 256 samples
                },
                {
                    "comment": "Measurement time after trigger",
                    "register": "8114",
                    "board": "1730",
                    "value": "20"
                    # 20hex = 32dec = 320 samples
                },
                {
                    "comment": "Interrupt status ID",
                    "register": "EF14",
                    "board": "-1",
                    "value": "55AA"
                    # Interrupt
                },
                {
                    "comment": "Interrupt event number",
                    "register": "EF18",
                    "board": "-1",
                    "value": "1"
                    # Interrupt when 1 event stored
                },
                {
                    "comment": "Trigger polarity",
                    "register": "8000",
                    "board": "1724",
                    "value": "20010"
                    # Set to 50 for negative, 10 for positive
                    # 20010 for ZLE enabled
                },
                {
                    "comment": "Trigger polarity",
                    "register": "8000",
                    "board": "1730",
                    "value": "10"
                    # Set to 50 for negative, 10 for positive
                    # Can also be set per-channel, in that case set to positive
                },
            ],
        }

        
    def set_register(self, reg, val, board = -1, comment=""):
        '''This sets one register value of a config dict.
        '''
        board = str(board)
        found_reg = False
        for i, reg_list in enumerate(self.cf['registers']):
            if reg_list['register'] == reg:
                if reg_list['board'] == board:
                    # register value exists, so change value
                    self.cf['registers'][i]['value'] = val
                    if self.verbose: print("Just set register %s (%s) with value %s for board %s" % (
                        self.cf['registers'][i]['register'],
                        self.cf['registers'][i]['comment'],
                        self.cf['registers'][i]['value'],
                        self.cf['registers'][i]['board'],
                                     ))
                    return
        reg_dict = {
            "comment" : comment,
            "register": reg,
            "board" : board,
            "value" : val}
        self.cf['registers'].append(reg_dict)
        if self.verbose: print("Just set register %s (%s) with value %s for board %s" % (
            self.cf['registers'][-1]['register'],
            self.cf['registers'][-1]['comment'],
            self.cf['registers'][-1]['value'],
            self.cf['registers'][-1]['board'],
                         ))
        return

    def get_register_value(self, reg, board):
        """ Get the value of the register on the board.
        """
        board = str(board)
        for reg_list in self.cf['registers']:
            if reg_list['register'] == reg:
                if reg_list['board'] == board:
                    return reg_list['value']
        if board != '-1':
            return self.get_register_value(reg, '-1')
        print('WARNING: register %s for board %s not found' % (reg, board))
        return
        


    def save_config(self):
        """ Dump the current config to a json file for daqcontrol to read.
        """
        config_path = os.path.join(self.output_dir, '%s.ini' % self.name)
        with open(config_path, 'w') as outfile:
            json.dump(cf, outfile, indent = 4)
        print('Output written to %s' % config_path)
        return
    
    def set_enabled_channels(self, board, channels = None):
        """ Set which channels are enabled.
        """
        if channels == -1:
            if self.verbose: print('Enabling all channels')
            channels = [i for i in range(8)]
        # Build enabled string
        bitstring = ''.join(['1' if i in channels else '0' for i in range(8)][::-1])
        hex_string = hex(int(bitstring, 2)).split(sep='x')[-1]
        self.set_register('8120', hex_string, board)
        return
    
    def set_trigger_channels(self, board, channels = None):
        """ Set the trigger enabled channels.
        NOTE: the 1730 channels are grouped in triggers, and this is not quite implemented yet.
        """
        board = str(board)
        if board == '-1':
            for board in self.boards:
                self.set_trigger_channels(board, channels = channels)
            return
        if channels == -1:
            print('Enabling all channels')
            channels = [i for i in range(8)]
        # Build enabled string
        bitstring = ''.join(['1' if i in channels else '0' for i in range(8)][::-1])
        hex_string = hex(int(bitstring, 2)).split(sep='x')[-1]
        current_value = self.get_register_value('810C', board)
        new_value = current_value[:-len(hex_string)] + hex_string
        self.set_register('810C', new_value, board)
        return
    
    def set_external_trigger(self, board, on=True):
        """ Enable or disable the external trigger on the board.
        """
        board = str(board)
        if board == '-1':
            for board in self.boards:
                self.set_external_trigger(board, on=on)
            return
        value = self.get_register_value('810C', board)
        value_list = list(value)
        if on:
            value_list[0] = 'C'
        else:
            value_list[0] = '5'
            print('WARNING unchecked')
        self.set_register('810C', ''.join(value_list), board)
        return
    
    def set_majority(self, board, majority):
        """ Set the majority level on the per-board trigger.
        Note that majority = number of channels in coincidence - 1.
        """
        value = self.get_register_value('810C', board)
        value_list = list(value)
        value_list[1] = str(majority)
        self.set_register('810C', ''.join(value_list), board)
        return
           
    def set_event_window(self, board, duration):
        '''Enter event window length in us
        Finds the buffer organization that spans the duration of duration, so the actual duration may be longer.
        Check ``set_event_window_custom`` for more options.
        '''
        board = str(board)
        if board == '1730':
            t0 = 5120 * 0.002
        elif board == '1724':
            t0 = 512 * 0.010
        elif board == '-1':
            for board in ['1724', '1730']:
                # Recursive definitions are awesome and like magic
                self.set_event_window(board, duration)
            return
        for buffer_div in range(11):
            t = t0 * 2**buffer_div
            if t >= duration:
                break
        setting = hex(10 - buffer_div).split(sep='x')[-1]
        if self.verbose: print('Using setting %s on %s, gives %.2f us' % (setting, board, t))
        self.set_register('800C', setting, board)
        return
        
    def set_trigger_position(self, board, t_after=None):
        ''' Set the trigger position in the window in units of us post-trigger.
        If ``t_after`` is not specified, the trigger is set in the middle of the window.
        '''
        board = str(board)
        if board == '-1':
            for board in [1730, 1724]:
                self.set_trigger_position(board, t_after = t_after)
            return
        nbuffer = self.get_register_value('800C', board)
        if not t_after:
            if board == '1724':
                ns_total = 512 * 2**(10 - int(nbuffer, 16))
                ns_post = ns_total // 4
                t_after = ns_post * 2 * 0.010
            elif board == '1730':
                ns_total = 5120 * 2**(10 - int(nbuffer, 16))
                ns_post = ns_total // 20
                t_after = ns_post * 10 * 0.002
        else:
            ns_post = int(t_after // 0.020)
            t_after = ns_post * 0.020

        if self.verbose: print('Setting post trigger window to %.3f us.' % t_after)
        value = hex(ns_post).split(sep='x')[-1]
        self.set_register('8114', value, board=board)
        return
    
    def set_event_window_custom(self, board, duration):
        """ Set a custom event window size in us. This will also set the buffer organization to the right value.
        """
        board = str(board)
        if board == '-1':
            for board in [1724, 1730]:
                self.set_event_window_custom(board, duration)
            return
        # First set buffer division containg the requested size
        self.set_event_window(board, duration)
        # After that, set curstom size register
        ns = int(duration / 0.020)
        if self.verbose: print('Setting board %s to custum window of %.3f us' % (board, ns * 0.02))
        value = hex(ns).split(sep='x')[-1]
        self.set_register('8020', value, board)

        return
    
    def set_trigger_level(self, board, channel, counts):
        """ Set the trigger level of the channel on the board in ADC counts.
        Note that counts may be positive or negative, and this changes the trigger polarity (i.e. negative
        number of counts means a trigger on the falling edge.)
        """
        board = str(board)
        baseline = self.baselines[board][channel]
        trigger_level = baseline + counts
        # The trigger polarity sets bit 15 on the register
        # 1 means negative. 
        # This is in logical OR with bit 6 of register 8000
        polarity = 1 if counts < 0 else 0
        
        value = hex(trigger_level + polarity * 2**15).split(sep='x')[-1]
       
        self.set_register('1%d80' % channel, value, )
        return
    
    def set_trigger_levels(self, board, counts):
        """ Set the trigger level in counts for all channels, for more documentation see ``set_trigger_level``
        """
        for channel in range(8):
            self.set_trigger_level(board, channel, counts)
        return
    
    def set_baseline(self, board, channel, baseline):
        """ Manually set the baselines used, per-channel. Preferred to build a baseline file.
        """
        board = str(board)
        self.baselines[board][channel] = baseline
        return
    
    def set_baselines(self, board, baselines):
        """ Manually set the baselines used. Preferred to build a baseline file.
        """
        board = str(board)
        self.baselines[board] = baselines
        return
    
    def set_zle_samples(self, n_samples):
        """ Set the number of samples post and pre ZLE-passing data.
        Currently set to the same before and after ZLE fires, and the same for each channel.
        """
        n_forward = n_samples
        n_backward = n_samples
        value = hex(n_forward + n_backward * 2 **16).split(sep='x')[-1]
        self.set_register('8028', value, board='1724')
        return
        
    def set_zle_level(self, channel, counts):
        """ Set the level (in counts, wrt the baseline) of ZLE on the V1724, per channel.
        """
        baseline = self.baselines['1724'][channel]
        zle_level = baseline + counts
        value = hex(zle_level).split(sep='x')[-1]
        self.set_register('1%d24' % channel, value, board='1724')
        return
    
    def set_zle_levels(self, counts):
        """ Set the level (in counts, wrt the baseline) of ZLE on the V1724.
        """
        for channel in range(8):
            self.set_zle_level(channel, counts)
        return
    
    def set_zle(self, on=True):
        """ Turn on or off the ZLE on all channels of the V1724.
        """
        value = '20010' if on else '10'
        set_register('8000', value, board='1724')
        return
        
    def set_channel_gain(self, channel, on=True):
        """ Set the channel gain to low or high on the V1730 digitizer.
        """
        value = '1' if on else '0'
        self.set_register('1%d28' % channel, value, board='1730')
        return
    
    def set_channel_gains(self, on=True):
        """ Set all channel gains to low or high on the V1730 digitizer.
        """
        for channel in range(8):
            self.set_channel_gain(channel, on=on)
        return

In [304]:
cf = Config('default')
cf.set_event_window(-1, 120.)
cf.set_trigger_position(-1)
cf.set_trigger_level(board='1730', channel=0, counts = 50)
cf.set_zle_levels(25)
cf.set_trigger_channels(board='1730', channels = [0])
cf.set_trigger_channels(board='1724', channels = [])

In [305]:
cf = Config('baselines')
cf.set_event_window_custom(-1, 2.) # 2 us on both boards
cf.set_trigger_position(-1, 1.)    # 1 us post trigger window
cf.set_enabled_channels(-1,-1)     # enable all channels on both boards
cf.set_trigger_channels(-1, [])
cf.set_external_trigger(-1, on=True)